In [1]:
'''
Author       : Aditya Jain
Date Started : 18th August, 2021
About        : This file does DL-based localization by directly reading video and outputing final video
'''
import torch
import torchvision.models as torchmodels
import torchvision
import os
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms
from PIL import Image
import cv2

In [23]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.get(name='MothAI',
                     subscription_id='1e5f7432-8004-48f7-a32d-668eee0f349e',
                     resource_group='MothProject'
                     )                    

In [24]:


model = Model(ws, 'DL_Localization_Model')

In [25]:
model

Model(workspace=Workspace.create(name='MothAI', subscription_id='1e5f7432-8004-48f7-a32d-668eee0f349e', resource_group='MothProject'), name=DL_Localization_Model, id=DL_Localization_Model:1, version=1, tags={}, properties={})

#### Model Loading

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# load a model pre-trained pre-trained on COCO
model       = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  # 1 class (person) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


MODEL_PATH  = '/home/mila/a/aditya.jain/logs/v1_localizmodel_2021-08-17-12-06.pt'
checkpoint  = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

In [29]:
model       = model.to(device)
model.eval()

DATA_PATH  = '/miniscratch/transit_datasets/restricted/inat_users/Localization/dl_test/orig_resized_images/'
SAVE_PATH  = '/miniscratch/transit_datasets/restricted/inat_users/Localization/dl_test/dl_bbox/'
SCORE_THR  = 0.99
image_list = os.listdir(DATA_PATH)

transform  = transforms.Compose([              
            transforms.ToTensor()])

for img in image_list:
    image_path = DATA_PATH + img
    image      = transform(Image.open(image_path))
    image_pred = torch.unsqueeze(image, 0).to(device)
    output     = model(image_pred)
    
    bboxes     = output[0]['boxes'][output[0]['scores'] > SCORE_THR]
    image_cv   = cv2.imread(image_path)
    
    for box in bboxes:
        box_numpy = box.detach().cpu().numpy()        
        cv2.rectangle(image_cv,(box_numpy[0], box_numpy[1]),(box_numpy[2], box_numpy[3]),(0,0,255),3)
     
    cv2.imwrite(SAVE_PATH + img, image_cv)    